In [94]:
import librosa
import numpy as n
import random
import soundfile as sf
import json
import csv
from pathlib import Path
from itertools import chain

def pitchify(y, sr, n_steps=None):
    if n_steps is None:
        n_steps = n.random.uniform(-3, 3)
    return librosa.effects.pitch_shift(y=y, sr=sr, n_steps= n_steps)

def stretch_time(y, rate=None):
    if rate is None:
        rate = n.random.uniform(0.4, 2.5)
    return librosa.effects.time_stretch(y=y, rate=rate)

def add_noise(y, noise_factor=None):
    if noise_factor is None:
        noise_factor = n.random.uniform(0.001, 0.01)
    noise_data = n.random.randn(len(y))
    return y + noise_factor * noise_data

def split(y, sr, min_duration=3, max_duration=10, n_chunks=5):
    chunks = []
    out = {}
    for i in range(n_chunks):
        max_samples = sr * max_duration
        start = n.random.randint(0, len(y) - max_samples) if len(y) - max_samples > 0 else 0
        chunk_duration = n.random.randint(sr * min_duration, sr * max_duration)
        end = min(start + chunk_duration, len(y))
        chunks.append(y[start:end])
        out[i] = (start, end)
    return chunks, out

def save_chunks(y_chunks, sr, base_name, path='chunks/'):
    chunk_paths = {}
    Path(path).mkdir(parents=True, exist_ok=True)
    for i, chunk in enumerate(y_chunks):
        filename = f'{base_name}_chunk_{i}.wav'
        full_path = Path(path) / filename
        sf.write(full_path, chunk, sr)
        chunk_paths[i] = str(full_path)
    return chunk_paths

def apply_effects(y, effects, base_name, sr=44100, path='chunks/'):
    file_names = []
    Path(path).mkdir(parents=True, exist_ok=True)
    for i, effect in enumerate(effects):
        y_mod = y.copy()
        if effect == "pitchify":
            y_mod = pitchify(y_mod, sr)
        elif effect == "time_stretch":
            y_mod = stretch_time(y_mod)
        elif effect == "noise":
            y_mod = add_noise(y_mod)
        else:
            continue
        filename = f"{base_name}_{effect}_{i}.wav"
        full_path = Path(path) / filename
        sf.write(full_path, y_mod, sr)
        file_names.append(str(full_path))
    return file_names

def ds_create(path: Path, output_json='labels.json', output_csv='labels.csv'):
    files = list(chain(path.glob('*.wav'), path.glob('*.mp3'), path.glob('*.flac')))
    if not files:
        print("No files found")
        return
    print(f"Found {len(files)} files")
    label = {}
    for idx, file in enumerate(files):
        available_effects = ["pitchify", "time_stretch", "noise"]
        n_effects = n.random.randint(1, 4)
        chosen_effects = list(n.random.choice(available_effects, n_effects, replace=False))
        y, sr = librosa.load(file, sr=None)
        chunks, time_stamps = split(y, sr, min_duration=3, max_duration=10, n_chunks=5)
        chunk_paths = []
        effect_file_outputs = []
        for i, (chunk, ts) in enumerate(zip(chunks, time_stamps.values())):
            chunk_base = f"{file.stem}_chunk{i}"
            chunk_out_path = save_chunks([chunk], sr, chunk_base)
            chunk_effect_files = apply_effects(chunk, chosen_effects, chunk_base, sr)
            chunk_paths.append(chunk_out_path)
            effect_file_outputs.append(chunk_effect_files)
        label[str(file)] = {
            "Index": idx,
            "File": str(file),
            "Effects Applied": chosen_effects,
            "Chunk Time Stamps (samples)": time_stamps,
            "Chunk Paths": chunk_paths,
            "Effect File Outputs": effect_file_outputs,
        }
    with open(output_json, 'w') as jf:
        json.dump(label, jf, indent=4)
    print(f"Labels saved to {output_json}")
    with open(output_csv, 'w', newline='') as cf:
        writer = csv.writer(cf)
        writer.writerow(["Index", "File", "Effects Applied", "Chunk Time Stamps (samples)", "Chunk Paths", "Effect File Outputs"])
        for data in label.values():
            writer.writerow([
                data["Index"],
                data["File"],
                '|'.join(data["Effects Applied"]),
                str(data["Chunk Time Stamps (samples)"]),
                str(data["Chunk Paths"]),
                str(data["Effect File Outputs"])
            ])
    print(f"Labels saved to {output_csv}")

ds_create(Path("ds/"))


Found 3 files
Labels saved to labels.json
Labels saved to labels.csv


In [88]:
ds_create(Path("ds/"))

Found 3 files


TypeError: pitch_shift() takes 1 positional argument but 3 were given

In [84]:
ls

chunks/      codes/       ds/          yfile.ipynb


In [89]:
import librosa
print(librosa.__file__)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/librosa/__init__.py
